In [63]:
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/copy-of-23-24-ml/train_data.csv
/kaggle/input/copy-of-23-24-ml/test_raw_data.jsonl
/kaggle/input/copy-of-23-24-ml/test_data.csv
/kaggle/input/copy-of-23-24-ml/train_raw_data.jsonl
/kaggle/input/copy-of-23-24-ml/train_targets.csv


# Dependecies

In [82]:
import pandas as pd
import optuna
import numpy as np
import joblib
import shap
import plotly.express as px
import json 
import seaborn as sns 
import lightgbm as lgb

from pathlib import Path
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, RepeatedStratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.manifold import SpectralEmbedding
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier

from optuna.samplers import RandomSampler

In [ ]:
# !pip install pycaret==3.3

In [ ]:
# from pycaret.classification import ClassificationExperiment

# Import data

In [ ]:
X = pd.read_csv('/kaggle/input/copy-of-23-24-ml/train_data.csv')
y = pd.read_csv('/kaggle/input/copy-of-23-24-ml/train_targets.csv')

# Pre-processing

In [ ]:
pd.set_option('display.max_columns', 500)

In [ ]:
X = pd.read_csv('/kaggle/input/copy-of-23-24-ml/train_data.csv')
y = pd.read_csv('/kaggle/input/copy-of-23-24-ml/train_targets.csv')

X_merged = X.merge(y, how='left', on= ['match_id_hash'])
y_merged = X_merged[['radiant_win']]

X_merged.drop(columns=['radiant_win', 'match_id_hash'], inplace=True)
X_merged= pd.get_dummies(X_merged, columns=['next_roshan_team'], drop_first=True)

print(X_merged.shape)

X_merged = X_merged[['next_roshan_team_Radiant', 'time_remaining', 'r3_gold', 'r4_gold',
       'r5_gold', 'r1_gold', 'r2_gold', 'd1_gold', 'd2_gold', 'r5_deaths',
       'd3_gold', 'd4_gold', 'd5_gold', 'r4_deaths', 'r1_deaths', 'r3_xp',
       'd3_roshans_killed', 'r1_xp', 'r3_deaths', 'r5_xp', 'r4_xp',
       'd2_deaths', 'r2_xp', 'd4_xp', 'd3_health', 'r2_deaths', 'd4_deaths',
       'd2_xp', 'd1_towers_killed', 'r2_level', 'd3_xp', 'd3_assists', 'd1_xp',
       'd2_rune_pickups', 'd4_assists', 'd3_deaths', 'd5_deaths', 'r1_assists',
       'd2_towers_killed', 'd5_roshans_killed', 'd5_xp', 'd2_assists',
       'd1_deaths', 'd5_towers_killed', 'r4_towers_killed', 'r5_assists',
       'd4_level', 'r4_assists', 'r4_health']]

X_merged.shape

# Pycaret

In [ ]:
# s = pycaret.classification.setup(X_merged, target = y_merged, session_id = 123, n_jobs=-1)

In [ ]:
# s.compare_models()

# Select features

In [ ]:
# import warnings
# warnings.filterwarnings("ignore")

# X = X_merged
# y = y_merged

# folds_scores = []
# folds_features = {}

# selected_features = X.columns

# while len(selected_features) > 20:
    
#     stratified_kfold_splitter = StratifiedKFold(n_splits=3, shuffle=True)
    
#     fold_feature_scores = []

#     for train_index, test_index in stratified_kfold_splitter.split(X, y):
#         X_train, X_test = X.iloc[train_index][selected_features], X.iloc[test_index][selected_features]
#         y_train, y_test = y.iloc[train_index].values.ravel(), y.iloc[test_index].values.ravel()  # Fix data shape
        
#         X_train = X_train.rank(axis=1)
#         X_test = X_test.rank(axis=1)
                
#         params = {
#         'learning_rate': [0.005, 0.1],
#         'n_estimators': [40, 100, 300],
#         'num_leaves': [6,8,12,16],
#         'colsample_bytree' : [0.65, 0.66],
#         'subsample' : [0.7,0.75],
#         'reg_alpha' : [1,1.2],
#         'reg_lambda' : [1,1.2,1.4],
#         }
                
#         model = LGBMClassifier(n_jobs = -1, verbose=-1)

#         skf = StratifiedKFold(n_splits=5, shuffle=True)

#         random_search = RandomizedSearchCV(
#             model,
#             param_distributions=params,
#             n_iter=10,
#             scoring='f1_micro',
#             n_jobs=-1,
#             cv=skf.split(X_train, y_train),
#             verbose=0)

#         random_search.fit(X_train, y_train)

#         best_params = random_search.best_params_

#         model = LGBMClassifier(**best_params, n_jobs =-1, verbose=-1)
#         model.fit(X_train, y_train)

#         model_score = f1_score(model.predict(X_test), y_test, average='micro')

#         print('Model f1:', model_score)

#         explainer = shap.TreeExplainer(model)
#         shap_values = explainer.shap_values(X_train)

#         fold_feature_score = pd.Series(
#             -np.abs(shap_values).mean(0).mean(0),
#                 index=X_test.columns
#         )

#         folds_scores.append({
#             'n_features': len(selected_features),
#             'f1_score': model_score
#         })

#         fold_feature_scores.append(fold_feature_score)
    
#     fold_feature_scores = pd.concat(fold_feature_scores, axis=1)
#     fold_feature_scores = fold_feature_scores.mean(axis=1)
    
#     fold_feature_scores = fold_feature_scores.sort_values()
    
#     selected_features = fold_feature_scores.iloc[:-1].index    
    
#     folds_features[len(selected_features)] = list(fold_feature_scores.iloc[:-5].index)
#     print(f'How many features are selected: {selected_features.shape[0]}')
    
# folds_scores = pd.DataFrame(folds_scores)

In [ ]:
# print(f'features: {selected_features}')

In [ ]:
# pd.Series(selected_features).to_csv('/kaggle/working/selected_features.csv')

# New dataframe 

In [65]:
X = pd.read_csv('/kaggle/input/copy-of-23-24-ml/train_data.csv')
y = pd.read_csv('/kaggle/input/copy-of-23-24-ml/train_targets.csv')

X_merged = X.merge(y, how='left', on= ['match_id_hash'])
y_merged = X_merged[['radiant_win']]

X_merged.drop(columns=['radiant_win', 'match_id_hash'], inplace=True)
X_merged= pd.get_dummies(X_merged, columns=['next_roshan_team'], drop_first=True)

print(X_merged.shape)
# selected_features = selected_features.to_list()
# X_merged = X_merged[selected_features]

# X_merged = X_merged[['d1_towers_killed', 'r3_gold', 'd5_gold', 'r2_gold', 'r4_gold',
#        'r5_gold', 'r1_gold', 'next_roshan_team_Radiant', 'd2_gold', 'd4_gold',
#        'r4_xp', 'r5_xp', 'd3_gold', 'r1_xp', 'd1_gold', 'r2_xp', 'd4_xp',
#        'd3_xp', 'd2_xp', 'd1_xp']]

X_merged = X_merged[['r1_xp', 'r2_xp', 'r3_xp', 'r4_xp', 'r5_xp',
                     'r1_gold', 'r2_gold', 'r3_gold','r4_gold', 'r5_gold',
                     'd1_xp', 'd2_xp', 'd3_xp', 'd4_xp', 'd5_xp',
                     'd1_gold', 'd2_gold', 'd3_gold','d4_gold', 'd5_gold',
                     'r1_kills','r1_deaths','r2_kills','r2_deaths','r3_kills','r3_deaths','r4_kills','r4_deaths','r5_kills','r5_deaths',
                     'd1_kills','d1_deaths','d2_kills','d2_deaths','d3_kills','d3_deaths','d4_kills','d4_deaths','d5_kills','d5_deaths']]
print(X_merged.shape)

X_merged['radiant_networth'] = X_merged.iloc[:,:10].sum(axis=1).astype('float')
X_merged['dire_networth'] = X_merged.iloc[:,10:20].sum(axis=1).astype('float')
X_merged['radiant_kd'] = np.log(X_merged.iloc[:,20:30:2].sum(axis=1).astype('float') / (X_merged.iloc[:,19:31:2].iloc[:,1:].sum(axis=1).astype('float') +0.01)+ 1)
X_merged['dire_kd'] = np.log(X_merged.iloc[:,30:39:2].sum(axis=1).astype('float') / (X_merged.iloc[:,29::2].iloc[:,1:].sum(axis=1).astype('float') +0.01)+ 1)
X_merged['kd_coeff'] = np.log(X_merged['radiant_kd'] / (X_merged['radiant_kd'] + X_merged['dire_kd'] + 1)+1)+1
X_merged['coeff'] = np.log(X_merged['radiant_networth'] /  X_merged['dire_networth'] * X_merged['kd_coeff'])
print(X_merged.shape)
# pd.concat((X_merged,y_merged),axis=1)

(31698, 249)
(31698, 40)
(31698, 46)


In [66]:
X_test = pd.read_csv('/kaggle/input/copy-of-23-24-ml/test_data.csv')
id = X_test['match_id_hash']
X_test = X_test[['r1_xp', 'r2_xp', 'r3_xp', 'r4_xp', 'r5_xp',
                     'r1_gold', 'r2_gold', 'r3_gold','r4_gold', 'r5_gold',
                     'd1_xp', 'd2_xp', 'd3_xp', 'd4_xp', 'd5_xp',
                     'd1_gold', 'd2_gold', 'd3_gold','d4_gold', 'd5_gold',
                     'r1_kills','r1_deaths','r2_kills','r2_deaths','r3_kills','r3_deaths','r4_kills','r4_deaths','r5_kills','r5_deaths',
                     'd1_kills','d1_deaths','d2_kills','d2_deaths','d3_kills','d3_deaths','d4_kills','d4_deaths','d5_kills','d5_deaths']]

X_test['radiant_networth'] = X_test.iloc[:,:10].sum(axis=1).astype('float')
X_test['dire_networth'] = X_test.iloc[:,10:20].sum(axis=1).astype('float')
X_test['radiant_kd'] = np.log(X_test.iloc[:,20:30:2].sum(axis=1).astype('float') / (X_test.iloc[:,19:31:2].iloc[:,1:].sum(axis=1).astype('float') +0.01)+ 1)
X_test['dire_kd'] = np.log(X_test.iloc[:,30:39:2].sum(axis=1).astype('float') / (X_test.iloc[:,29::2].iloc[:,1:].sum(axis=1).astype('float') +0.01)+ 1)
X_test['kd_coeff'] = np.log(X_test['radiant_kd'] / (X_test['radiant_kd'] + X_test['dire_kd'] + 1)+1)+1
X_test['coeff'] = np.log(X_test['radiant_networth'] /  X_test['dire_networth'] * X_test['kd_coeff'])
X_test

,r1_xp,r2_xp,r3_xp,r4_xp,r5_xp,r1_gold,r2_gold,r3_gold,r4_gold,r5_gold,...,d4_kills,d4_deaths,d5_kills,d5_deaths,radiant_networth,dire_networth,radiant_kd,dire_kd,kd_coeff,coeff
0,533,478,130,506,344,543,399,304,389,402,...,1,0,0,0,4028.0,7032.0,0.000000,0.000142,1.000000,-0.557201
1,1471,3602,4811,1228,3551,1613,2816,4017,1558,3344,...,0,2,0,1,28011.0,23144.0,1.382556,0.000043,1.457597,0.567651
2,1351,1693,3210,1964,1544,1404,1840,1204,1724,1907,...,2,1,0,1,17841.0,19983.0,0.404634,0.000200,1.253114,0.112249
3,3088,2933,5286,2599,3814,2306,3872,4293,2695,3994,...,3,0,0,2,34880.0,32753.0,0.559081,0.000214,1.306416,0.330206
4,5081,5800,3060,2710,3373,3917,3961,2861,3037,3342,...,2,2,0,3,37142.0,34615.0,1.176927,0.000087,1.432182,0.429660
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7972,2794,3138,3071,1782,2446,2621,1575,3588,2049,2233,...,0,2,0,0,25297.0,20413.0,1.601486,0.000049,1.479702,0.606354
7973,9094,26935,20735,17349,25145,7223,23040,17768,13784,19299,...,9,6,6,7,180372.0,196092.0,0.639560,0.000229,1.329322,0.201107
7974,7847,10815,9270,19257,10367,6283,8838,8530,14815,9864,...,10,5,1,4,105886.0,116927.0,0.465973,0.000274,1.275964,0.144515
7975,19847,18449,27114,13382,26950,14375,12448,21451,8134,32403,...,5,6,9,7,194553.0,188625.0,0.717712,0.000207,1.349092,0.330376


In [ ]:
# sns.catplot(data=pd.concat((X_merged,y_merged),axis=1),y='coeff', x='radiant_win', kind='violin')

In [ ]:
# sns.boxplot(y=X_merged['kd_coeff'], x=y_merged['radiant_win'])

In [ ]:
# sns.boxplot(y=X_merged['kd_coeff'], x=y_merged['radiant_win'])

In [ ]:
# sns.boxplot(y=X_merged['kd_coeff'], x=y_merged['radiant_win'])

In [57]:
train_X, test_X, train_y, test_y = train_test_split(X_merged,
                                                    y_merged,
                                                    test_size=0.2,
                                                    random_state = 42)

print(X_train.shape, X_test.shape)

(25358, 47) (7977, 46)


# Dimensionality reduction

In [ ]:
# pca = PCA(n_components=20)
# components = pca.fit_transform(X_merged)

# fig = px.scatter_matrix(
#     components,
#     color=y_merged['radiant_win'],
#     dimensions=range(4)
# )
# fig.update_traces(diagonal_visible=False)
# fig.show()

In [ ]:
# embedding = SpectralEmbedding(n_components=2)
# components = embedding.fit_transform(X_merged)

# fig = px.scatter(
#     components,
#     color=y_merged['radiant_win']
# )
# fig.update_traces(diagonal_visible=False)
# fig.show()

# Optuna

In [ ]:
# class ObjectiveCV(object):
#     def __init__(self, X_data, y_data, multilabel_data):
#         self.X_data = X_data
#         self.y_data = y_data
#         self.multilabel_data = multilabel_data

#     def __call__(self, trial):

#         params = {
#             "objective": "binary",
#             "metric": "auc",
#             "boosting_type": "gbdt",  
#             "verbosity": -1,
#             "bagging_freq": trial.suggest_int("bagging_freq", 1, 9),
#             "n_estimators": trial.suggest_int("n_estimators", 50, 2500),  # n_trees
#             "lambda_l1": trial.suggest_float("lambda_l1", 1e-2, 10.0, log=True),
#             "lambda_l2": trial.suggest_float("lambda_l2", 1e-2, 10.0, log=True),
#             "feature_fraction": trial.suggest_float("feature_fraction", 0.05, 1.0),
#             "bagging_fraction": trial.suggest_float("bagging_fraction", 0.05, 1.0),
#             "min_child_samples": trial.suggest_int("min_child_samples", 5, 5000),
#             "max_depth": trial.suggest_int("max_depth", 3, 17),
#             "learning_rate": trial.suggest_float("learning_rate", 0.0001, 0.3),
#         }

#         params["num_leaves"] = 2 ** params["max_depth"]

#         cv = StratifiedKFold(
#             n_splits=4, 
#             shuffle=True,
#             random_state=42
#         )

#         result = []
#         for train_index, test_index in cv.split(self.X_data, self.y_data):
#             X_train, X_test = (
#                 self.X_data.iloc[train_index],
#                 self.X_data.iloc[test_index],
#             )
#             y_train, y_test = (
#                 self.y_data.iloc[train_index],
#                 self.y_data.iloc[test_index],
#             )

#             model = LGBMClassifier(**params)
#             model.fit(X_train.astype(float), y_train)
#             y_pred = model.predict(X_test)
#             accuracy = accuracy_score(y_test, y_pred)

#             result.append(accuracy)

#         mean_accuracy = np.mean(result)
        
#         return mean_accuracy

In [ ]:
# import warnings
# warnings.filterwarnings("ignore")

# objective_cv = ObjectiveCV(X_train, y_train, y_train)

# study_cv = optuna.create_study(direction="maximize")
# # study_cv.optimize(objective_cv, n_trials=20, n_jobs=-1, gc_after_trial=True)
# study_cv.optimize(objective_cv, timeout=28800) 
# best_params_cv = study_cv.best_params
# print("Best Hyperparameters with OptunaCV:", best_params_cv)
# print(study.best_value)

# Optuna 2

In [ ]:
# dtrain = lgb.Dataset(X_train, label=y_train,params={'verbose': -1})
# val_set = lgb.Dataset(X_test, label=y_test)
# rkf = RepeatedStratifiedKFold(n_splits=10, n_repeats=10, random_state=42)
# # We will track how many training rounds we needed for our best score.
# # We will use that number of rounds later.
# best_score = 0.5
# training_rounds = 10000

# # Declare how we evaluate how good a set of hyperparameters are, i.e.
# # declare an objective function.
# def objective(trial):
#     callbacks = [
#         lgb.early_stopping(stopping_rounds=250,verbose=False),  # Adjust this parameter as needed
#         lgb.reset_parameter(learning_rate=[0.005] * 200 + [0.001] * 9800)  # Existing callback
#     ]
#     # Specify a search space using distributions across plausible values of hyperparameters.
#     param = {
#         "objective": "binary",
#         "metric": "auc",
#         "verbose": -1,
#         "boosting_type": "gbdt",  
#         "silent" : True,
#         "seed": 42,
#         'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
#         'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
#         'num_leaves': trial.suggest_int('num_leaves', 2, 512),
#         'feature_fraction': trial.suggest_uniform('feature_fraction', 0.1, 1.0),
#         'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.1, 1.0),
#         'bagging_freq': trial.suggest_int('bagging_freq', 0, 15),
#         "max_depth": trial.suggest_int("max_depth", 3, 17),
#     }

#     # Run LightGBM for the hyperparameter values
#     lgbcv = lgb.train(param,
#                    dtrain,
# #                    categorical_feature=ids_of_categorical,
# #                    folds=rkf,   
#                    valid_sets=[val_set],
#                    num_boost_round=10000,                    
#                    callbacks=callbacks
#                   )
#     preds=lgbcv.predict(X_test)
#     pred_labels = np.rint(preds)
#     accuracy = accuracy_score(y_test, pred_labels)

#     return accuracy

# # Suppress information only outputs - otherwise optuna is 
# # quite verbose, which can be nice, but takes up a lot of space
# import logging
# logging.getLogger("optuna").setLevel(logging.WARNING)
# logging.getLogger("lightgbm").setLevel(logging.WARNING)

# # optuna.logging.set_verbosity(optuna.logging.WARNING)
# import warnings
# warnings.filterwarnings("ignore")
# # We search for another 4 hours (3600 s are an hours, so timeout=14400).
# # We could instead do e.g. n_trials=1000, to try 1000 hyperparameters chosen 
# # by optuna or set neither timeout or n_trials so that we keep going until 
# # the user interrupts ("Cancel run").
# study = optuna.create_study(direction='maximize')  
# study.optimize(objective, timeout=28800) 


In [ ]:
# print(study.best_params)
# # Classification error
# print(study.best_value)

# Optuna 3

In [ ]:
bounds_LGB = {
    'num_leaves': (31, 500), 
    'min_data_in_leaf': (20, 200),
    'bagging_fraction' : (0.1, 0.9),
    'feature_fraction' : (0.1, 0.9),
    #'learning_rate': (0.01, 0.3),
    'min_child_weight': (0.00001, 0.01),   
    'reg_alpha': (1, 2), 
    'reg_lambda': (1, 2),
    'max_depth':(-1,50),
}

In [ ]:
def objective(trial,data=X_merged,target=y_merged['radiant_win']):
    
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.2,random_state=42)

    # Specify a search space using distributions across plausible values of hyperparameters.
    param = {
        "objective": "binary",
        "metric": "auc",
        "verbose": -1,
        "boosting_type": "gbdt",  
        "silent" : True,
        "seed": 42,
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.1, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.1, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 0, 15),
        "max_depth": trial.suggest_int("max_depth", 3, 17),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 5000),
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.3),
        "n_estimators": trial.suggest_int("n_estimators", 50, 2500),  # n_trees
        'num_leaves': (31, 500), 
        'min_data_in_leaf': (20, 200),
    }
    param["num_leaves"] = 2 ** param["max_depth"]
    model = LGBMClassifier(**param)  
    
    model.fit(train_x,train_y,eval_set=[(test_x,test_y)])
    
    preds = model.predict(test_x)
    
    accuracy = accuracy_score(test_y, preds)
    
    return accuracy

In [ ]:
import warnings
warnings.filterwarnings("ignore")
# optuna.logging.set_verbosity(optuna.logging.WARNING)

study = optuna.create_study(direction='maximize')  
study.optimize(objective, timeout=28800) 

print(study.best_params)
# Classification error
print(study.best_value)

In [ ]:
# def get_models():
#     models = dict()
#     for d in range(1,8):
#     # define the pipeline
#         trans = PolynomialFeatures(degree=d)
#         model = LGBMClassifier(verbose=-1)
#         models[str(d)] = Pipeline(steps=[('t', trans), ('m', model)])
#     return models

In [ ]:
# # evaluate a give model using cross-validation
# def evaluate_model(model, X, y):
#     cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
#     scores = cross_val_score(model, X, y, scoring='f1_micro', cv=cv, n_jobs=-1, error_score='raise')
#     return scores

# # get the models to evaluate
# models = get_models()

# # evaluate the models and store results
# results, names = list(), list()
# for name, model in models.items():
#     scores = evaluate_model(model, X_merged, y_merged['radiant_win'])
#     results.append(scores)
#     names.append(name)
#     print('>%s %.3f (%.3f)' % (name, np.mean(scores), np.std(scores)))

# # plot model performance for comparison
# pyplot.boxplot(results, labels=names, showmeans=True)
# pyplot.show()
# #Box Plots of Degree for the Polynomial degree

In [ ]:
# best_params_cv = study.best_params
# best_params_cv['verbose'] = -1

# best_model = LGBMClassifier(**best_params_cv)
# best_model.fit(X_train, y_train)

# y_train_predict = best_model.predict(X_train)
# y_test_predict = best_model.predict(X_test)

# print(f'train {classification_report(y_train, y_train_predict)}')
# print(f'test {classification_report(y_test, y_test_predict)}')
# print(f'accuracy {accuracy_score(y_test, y_test_predict)}')

In [30]:
best_params_cv = {'lambda_l1': 8.26224894128066e-05, 'lambda_l2': 0.013199593874817058, 'num_leaves': 129, 'feature_fraction': 0.6232453438530957, 'bagging_fraction': 0.771295169733037, 'bagging_freq': 9, 'max_depth': 7, 'min_child_samples': 84, 'learning_rate': 0.044319664340884385}
best_params_cv['verbose'] = -1

best_model = LGBMClassifier(**best_params_cv)
best_model.fit(X_merged, y_merged['radiant_win'])

preds = best_model.predict(X_merged)

print(f'train {classification_report(y_merged, preds)}')
print(f'test {classification_report(y_merged, preds)}')
print(f'accuracy {accuracy_score(y_merged, preds)}')

train               precision    recall  f1-score   support

       False       0.76      0.72      0.74     15028
        True       0.76      0.80      0.78     16670

    accuracy                           0.76     31698
   macro avg       0.76      0.76      0.76     31698
weighted avg       0.76      0.76      0.76     31698

test               precision    recall  f1-score   support

       False       0.76      0.72      0.74     15028
        True       0.76      0.80      0.78     16670

    accuracy                           0.76     31698
   macro avg       0.76      0.76      0.76     31698
weighted avg       0.76      0.76      0.76     31698

accuracy 0.7610259322354723


In [38]:
test_pred = pd.DataFrame(best_model.predict_proba(X_test)[:,1])
res = pd.concat((id,test_pred),axis=1)
res.rename(columns={'radiant_win':'radiant_win_prob'},inplace=True)
res.to_csv('results.csv', index=False)

In [50]:
res

,match_id_hash,radiant_win
0,a400b8f29dece5f4d266f49f1ae2e98a,0.312171
1,46a0ddce8f7ed2a8d9bd5edcbb925682,0.744998
2,b1b35ff97723d9b7ade1c9c3cf48f770,0.354749
3,ab3cc6ccac661a1385e73a2e9f21313a,0.602495
4,54aaab1cb8cc5df3c253641618673266,0.619797
...,...,...
7972,03f683fed8cf6f03b735e2ee2a942698,0.787842
7973,06e2b42495855647830ccc6d466f913c,0.354599
7974,defd0caeed6ea83d7d5fbdec013fe7d1,0.228531
7975,47ad6454ede66c1c78fdaa9391dfc556,0.668445


In [75]:
# Get the train and test data for the training sequence
train_data = lgb.Dataset(train_X, label=train_y['radiant_win'])
test_data = lgb.Dataset(test_X, label=test_y['radiant_win'])
evals={}
# Set the parameters for training
parameters = {'verbosity': -1, 'lambda_l1': 8.26224894128066e-05, 'lambda_l2': 0.013199593874817058, 'num_leaves': 129, 'feature_fraction': 0.6232453438530957, 'bagging_fraction': 0.771295169733037, 'bagging_freq': 9, 'max_depth': 7, 'min_child_samples': 84, 'learning_rate': 0.044319664340884385}
# Train the classifier
classifier = lgb.train(parameters,
                       train_data,
                       valid_sets= test_data,
                       num_boost_round=10000)



In [76]:
print(roc_auc_score(test_y['radiant_win'], classifier.predict(test_X)))
# Make predictions
predictions = pd.DataFrame(classifier.predict(X_test))
# Create submission file
my_pred_lgbm = pd.concat((id,predictions),axis=1)
my_pred_lgbm.rename(columns={0:'radiant_win_prob'},inplace=True)
# Create CSV file
my_pred_lgbm.to_csv('pred_lgbm.csv', index=False)

0.7662829084314665


In [77]:
my_pred_lgbm

,match_id_hash,radiant_win_prob
0,a400b8f29dece5f4d266f49f1ae2e98a,0.225780
1,46a0ddce8f7ed2a8d9bd5edcbb925682,0.987348
2,b1b35ff97723d9b7ade1c9c3cf48f770,0.276003
3,ab3cc6ccac661a1385e73a2e9f21313a,0.484813
4,54aaab1cb8cc5df3c253641618673266,0.351318
...,...,...
7972,03f683fed8cf6f03b735e2ee2a942698,0.628228
7973,06e2b42495855647830ccc6d466f913c,0.427949
7974,defd0caeed6ea83d7d5fbdec013fe7d1,0.224307
7975,47ad6454ede66c1c78fdaa9391dfc556,0.633994


# Meta-classifier

In [79]:
def meta_classifier(base_clfs, final_classifier, X_train, X_test, y_train, cv):
    """
    Meta classifier prediction using stacking. 
    Input:
    :param base_clfs: list,  base classifiers which will be stacked together.
    :param final_classifier: estimator, a classifier which will be used to combine the base estimators. 
    :param X_train: numpy array or pandas table, train set.
    :param X_test: numpy array or pandas table, target for train set.
    :param X_train: numpy array or pandas table, test set.
    :param cv: number of cross-validation folds.
    
    Output:
    :param y_pred: numpy array or pandas table, prediction of meta classifier using stacking on test set.
    :param final_classifier(optional): estimator, trained final_calssifier.
    
    
    More details https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.StackingClassifier.html
    
    """
    meta_features = list()
    meta_features_test = list()
    for clf in base_clfs:
        meta_features.append(cross_val_predict(clf, X_train, y_train, cv = cv))
        clf.fit(X_train, y_train)
        meta_features_test.append(clf.predict(X_test))

    final_classifier.fit(np.array(meta_features).T, y_train)
    meta_clf_test_predict = final_classifier.predict_proba(np.array(meta_features_test).T)[:,1]

    return meta_clf_test_predict, final_classifier

In [87]:
models = [('LightGBM', LGBMClassifier(**best_params_cv)),
           ('LightGBM', LGBMClassifier(**best_params_cv)),
           ('LightGBM', LGBMClassifier(**best_params_cv)),
           ('LightGBM', LGBMClassifier(**best_params_cv)),
           ('GBclf', GradientBoostingClassifier())]

for model in models:
    model[1].fit(train_X, train_y['radiant_win'])

for model in models:
    f1 = f1_score(test_y, model[1].predict(test_X))
    accuracy = accuracy_score(test_y, model[1].predict(test_X))
    print(f'f1 of {model[0]} on test: {f1:.3f}')
    print(f'accuracy of {model[0]} on test: {accuracy:.3f}')

models_no_labels = [model[1] for model in models]
meta_classificator = LGBMClassifier(**best_params_cv)
y_test_meta, logistic_regression_meta= meta_classifier(models_no_labels, meta_classificator, train_X, test_X, train_y['radiant_win'], 3)

f1 of LightGBM on test: 0.734
accuracy of LightGBM on test: 0.713
f1 of LightGBM on test: 0.734
accuracy of LightGBM on test: 0.713
f1 of LightGBM on test: 0.734
accuracy of LightGBM on test: 0.713
f1 of LightGBM on test: 0.734
accuracy of LightGBM on test: 0.713
f1 of GBclf on test: 0.730
accuracy of GBclf on test: 0.712


ValueError: Classification metrics can't handle a mix of binary and continuous targets

In [90]:
# accuracy = accuracy_score(test_y, y_test_meta)
# f1 = f1_score(test_y, y_test_meta)
roc = roc_auc_score(test_y, y_test_meta)
# print(f'f1 of metaclassifier {meta_classificator.__class__.__name__} on original datasets features on test: {f1:.3f}')
# print(f'accuracy of metaclassifier {meta_classificator.__class__.__name__} on original datasets features on test: {accuracy:.3f}')
print(f'roc auc metaclassifier {meta_classificator.__class__.__name__} on original datasets features on test: {accuracy:.3f}')

roc auc metaclassifier LGBMClassifier on original datasets features on test: 0.712


In [ ]:
# # save model
# joblib.dump(best_model, '/kaggle/working/lgb_optuna.pkl')


In [ ]:
# # load model
# best_model_pkl = joblib.load('/kaggle/working/lgb_optuna.pkl')